<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Investigaci-n-de-operaciones/blob/main/TareaIO4/Modelo_de_transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [90]:
class ModeloTransporte:
  def asignar(Ma,oferta,demanda, i,j):
    if oferta[i]==0 and demanda[i]==0:
      return None
    #Se encarga de asignar la oferta, revisa si se puede colocar la oferta disponible o si no coloca la demanda disponible
    if oferta[i]<=demanda[j]: 
      Ma[i][j]=oferta[i]
    else:
    #Elimina la demanda columna oferta de sus vectores
      Ma[i][j]=demanda[j]
    oferta[i]-=Ma[i][j]
    demanda[j]-=Ma[i][j]

  def eliminar(K,oferta,demanda):
    for i in range(len(oferta)):
      if(oferta[i]==0):
        K[i,:]=np.ones(len(K[i,:]))*1e5
    for i in range(len(demanda)):
      if(demanda[i]==0):
        K[:,i]=np.ones(len(K[:,i]))*1e5

  def calcular(C,M):#La suma de la diagonal de la matriz de resultados columna la matriz de costos es igual a calcular el costo
    S=np.array(C).T@np.array(M)
    sol=sum([S[i][i] for i in range(len(S))])
    return sol

  def norOeste(C,oferta,demanda):
    #Construye la matriz de resultados
    M=[[0 for i in C[0]] for i in C]
    for i in range(len(C)):
      for j in range(len(C[0])):
        ModeloTransporte.asignar(M,oferta,demanda,i,j)
    return M,ModeloTransporte.calcular(C,M)
    
  def minimoCosto(C,oferta,demanda,M=None,K=None):
    #Hace una copia de la matriz de costos
    if M==None:
      K=np.array(C)
      M=[[0 for i in C[0]] for i in K]
    
    minimos=[np.min(i) for i in K]
    fila=np.argmin(minimos)
    columna=[np.argmin(i) for i in K][fila]

    ModeloTransporte.asignar(M,oferta,demanda,fila,columna)
    ModeloTransporte.eliminar(K,oferta,demanda)

    if sum(demanda)+sum(oferta)!=0:
      ModeloTransporte.minimoCosto(C,oferta,demanda,M,K)
    
    return M,ModeloTransporte.calcular(C,M)
    

  def vogel(C,oferta,demanda,M=None,K=None):
    if M==None:
      M=[[0 for i in C[0]] for i in C]
      K=np.array(C)
    pf=[abs(np.sort(K[:,i])[0]-np.sort(K[:,i])[1]) for i in range(len(demanda))]
    pc=[abs(np.sort(K[i,:])[0]-np.sort(K[i,:])[1]) for i in range(len(oferta))]
    if np.max(pf)>np.max(pc):
      fila,columna=np.argmin(K[:,np.argmax(pf)]),np.argmax(pf)
    elif np.max(pf)<=np.max(pc):
      fila,columna=np.argmax(pc),np.argmin(K[np.argmax(pc),:])

    ModeloTransporte.asignar(M,oferta,demanda,fila,columna)
    ModeloTransporte.eliminar(K,oferta,demanda)
    if sum(demanda)+sum(oferta)!=0:
      ModeloTransporte.vogel(C,oferta,demanda,M,K)
    return M,ModeloTransporte.calcular(C,M)

  def solve(C,oferta,demanda):
    _,s1=ModeloTransporte.norOeste(np.array(C),np.array(oferta),np.array(demanda))
    _,s2=ModeloTransporte.minimoCosto(np.array(C),np.array(oferta),np.array(demanda))
    _,s3=ModeloTransporte.vogel(np.array(C),np.array(oferta),np.array(demanda))
    return np.min([s1,s2,s3])
  

In [91]:
C=[[10,2,20,11],
   [12,7,9,20],
   [4,14,16,18]]
oferta=[15,25,10]
demanda=[5,15,15,15]
ModeloTransporte.solve(C,oferta,demanda)

475

In [94]:
C=[[10,2,20,11],
   [12,7,9,20],
   [4,14,16,18]]
oferta=[15,25,10]
demanda=[5,15,15,15]
ModeloTransporte.vogel(C,oferta,demanda)

([[0, 15, 0, 0], [0, 0, 15, 10], [5, 0, 0, 5]], 475)